In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Experimental']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
11,12,Love in the Time of Lexapro EP,Oneohtrix Point Never,"daniel lopatin’s , project is a study in futur...",Warp,6.5,November,26,2018,Experimental,NaN,NaN,NaN
13,14,From the Series of Songs “Altogether Unaccompa...,Helena Deland,"helena deland’s , bears an unusual title: exac...",Luminelle,7.8,October,23,2018,Experimental,NaN,NaN,NaN
86,87,Warzone,Yoko Ono,"“people of america, when will we learn?” yoko ...",Chimera,6.3,October,23,2018,Experimental,NaN,NaN,NaN
87,88,Spoken By the Other EP,"Wolfgang Tillmans,Powell",’ work is a reminder that no freedom is a give...,XL,6.8,November,30,2018,Experimental,Electronic,NaN,NaN
107,108,EP ONE,Galya Bisengalieva,"even if you have never heard her name, you mig...",Nomad,7.6,December,8,2018,Experimental,NaN,NaN,NaN
109,110,Hence,"Oren Ambarchi,Jim O’Rourke","in the early 21st century, , and , were workin...",Editions Mego,7.4,December,8,2018,Experimental,NaN,NaN,NaN
113,114,Throne,Heather Leigh,"for more than 20 years, the improvisational mu...",Editions Mego,7.7,December,7,2018,Experimental,NaN,NaN,NaN
134,135,Moriyama EP,Foodman,"in the most frenetic footwork tracks, a curiou...",Palto Flats,7.5,November,27,2018,Experimental,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['it', 'might', 'seem', 'for', 'drummer', 'to', 'embrace', 'electronic', 'music', 'after', 'all', 'some', 'say', 'while', 'others', 'still', 'insist', 'that', 'groove', 'boxes', 'will', 'even', 'if', 'that', 'idea', 'is', 'less', 'prevalent', 'than', 'it', 'was', 'but', 'electronic', 'music', 'is', 'fundamentally', 'time', 'based', 'art', 'and', 'no', 'one', 'understands', 'the', 'intricacies', 'of', 'clockwork', 'better', 'than', 'drummers', 'species', 'with', 'heightened', 'perception', 'of', 'the', 'relationship', 'between', 'muscle', 'and', 'millisecond', 'raft', 'of', 'drummers', 'have', 'recently', 'fused', 'their', 'instrumental', 'practice', 'with', 'electronic', 'processing', 'putting', 'percussive', 'spin', 'on', 'electro', 'acoustic', 'composition', 'from', 'the', 'laptop', 'aided', 'layering', 'of', 'and', 'to', 'the', 'drill', 'bass', 'mechanics', 'of', 'add', 'to', 'that', 'roster', 'the', 'impeccably', 'named', 'member', 'of', 'cloud', 'becomes', 'your', 'hand', 'and', 

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['it', 'might', 'seem', 'for', 'drummer', 'to', 'embrace', 'electronic', 'music', 'after', 'all', 'some', 'say', 'while', 'others', 'still', 'insist', 'that', 'groove', 'boxes', 'will', 'even', 'if', 'that', 'idea', 'is', 'less', 'prevalent', 'than', 'it', 'was', 'but', 'electronic', 'music', 'is', 'fundamentally', 'time', 'based', 'art', 'and', 'no', 'one', 'understands', 'the', 'intricacies', 'of', 'clockwork', 'better', 'than', 'drummers', 'species', 'with', 'heightened', 'perception', 'of', 'the', 'relationship', 'between', 'muscle', 'and', 'millisecond', 'raft', 'of', 'drummers', 'have', 'recently', 'fused', 'their', 'instrumental', 'practice', 'with', 'electronic', 'processing', 'putting', 'percussive', 'spin', 'on', 'electro', 'acoustic', 'composition', 'from', 'the', 'laptop', 'aided', 'layering', 'of', 'and', 'to', 'the', 'drill', 'bass', 'mechanics', 'of', 'add', 'to', 'that', 'roster', 'the', 'impeccably', 'named', 'member', 'of', 'cloud', 'becomes', 'your', 'hand', 'and', '

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['may', 'seem', 'drummer', 'embrace', 'electronic', 'music', 'say', 'other', 'still', 'insist', 'groove', 'box', 'even', 'idea', 'less', 'prevalent', 'electronic', 'music', 'fundamentally', 'time', 'base', 'art', 'one', 'understand', 'intricacy', 'clockwork', 'good', 'drummer', 'specie', 'heighten', 'perception', 'relationship', 'muscle', 'millisecond', 'raft', 'drummer', 'recently', 'fuse', 'instrumental', 'practice', 'electronic', 'processing', 'put', 'percussive', 'spin', 'electro', 'acoustic', 'composition', 'laptop', 'aid', 'layering', 'drill', 'bass', 'mechanic', 'add', 'roster', 'impeccably', 'name', 'member', 'cloud', 'become', 'hand', 'collaborator', 'ranaldo', 'cline', 'stardrum', 'second', 'album', 'slip', 'slide', 'propulsive', 'stickwork', 'program', 'sequencing', 'glower', 'metallic', 'drone', 'never', 'settle', 'terrain', 'long', 'often', 'dissolve', 'post_punk', 'mulch', 'obscure', 'origin', 'stardrum', 'sound', 'wipe', 'away', 'dirt', 'leave', 'signature', 'sharp', 'r

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)]]


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"metal" + 0.034*"attention" + 0.033*"element" + 0.029*"black" + '
  '0.028*"color" + 0.027*"white" + 0.027*"source" + 0.026*"image" + '
  '0.026*"film" + 0.025*"bear"'),
 (1,
  '0.036*"previous" + 0.030*"sonic" + 0.029*"deliver" + 0.028*"chaos" + '
  '0.026*"experience" + 0.026*"duo" + 0.022*"studio" + 0.022*"largely" + '
  '0.021*"remain" + 0.021*"describe"'),
 (2,
  '0.112*"jazz" + 0.040*"london" + 0.036*"share" + 0.035*"war" + 0.035*"push" '
  '+ 0.030*"wave" + 0.027*"star" + 0.024*"label" + 0.023*"scene" + '
  '0.021*"future"'),
 (3,
  '0.051*"dead" + 0.049*"black" + 0.048*"industrial" + 0.038*"distance" + '
  '0.031*"familiar" + 0.031*"dream" + 0.030*"image" + 0.026*"trace" + '
  '0.025*"float" + 0.024*"act"'),
 (4,
  '0.026*"score" + 0.015*"low" + 0.014*"career" + 0.014*"film" + '
  '0.013*"bright" + 0.013*"tune" + 0.012*"japanese" + 0.012*"sort" + '
  '0.011*"soundtrack" + 0.010*"rest"'),
 (5,
  '0.026*"young" + 0.019*"add" + 0.016*"layer" + 0.013*"feeling" + '
  '

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.0137000056789205

Coherence Score:  0.203847635049443


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.040400  0.028188       1        1  28.337410
5     -0.077002 -0.029137       2        1  24.041332
7     -0.088091  0.153770       3        1   9.643888
6     -0.000666  0.116174       4        1   8.515330
9      0.221720  0.030139       5        1   7.659015
0     -0.166233 -0.101162       6        1   5.993527
1      0.014985  0.111267       7        1   5.696650
2      0.141610 -0.232958       8        1   4.930203
3     -0.160814 -0.100959       9        1   3.956883
8      0.074090  0.024676      10        1   1.225766, topic_info=    Category       Freq         Term      Total  loglift  logprob
234  Default  45.000000         jazz  45.000000  30.0000  30.0000
267  Default  23.000000     anything  23.000000  29.0000  29.0000
76   Default  37.000000     original  37.000000  28.0000  28.0000
67   Default  33.000000      element  33.000000  27.0000  27.0000
131  Default  39.000000        black  39.000000  26.0000  26.0000
278  Default  24.000000        allow  24.000000  25.0000  25.0000
220  Default  31.000000         dead  31.000000  24.0000  24.0000
47   Default  31.000000         hour  31.000000  23.0000  23.0000
172  Default  36.000000       remain  36.000000  22.0000  22.0000
22   Default  17.000000       mirror  17.000000  21.0000  21.0000
24   Default  23.000000        organ  23.000000  20.0000  20.0000
147  Default  33.000000        metal  33.000000  19.0000  19.0000
17   Default  22.000000          hum  22.000000  18.0000  18.0000
42   Default  30.000000     ensemble  30.000000  17.0000  17.0000
85   Default  30.000000        share  30.000000  16.0000  16.0000
26   Default  25.000000      pattern  25.000000  15.0000  15.0000
210  Default  35.000000       studio  35.000000  14.0000  14.0000
88   Default  24.000000        state  24.000000  13.0000  13.0000
282  Default  25.000000       london  25.000000  12.0000  12.0000
169  Default  24.000000       phrase  24.000000  11.0000  11.0000
171  Default  28.000000         push  28.000000  10.0000  10.0000
264  Default  39.000000         film  39.000000   9.0000   9.0000
63   Default  30.000000         wave  30.000000   8.0000   8.0000
62   Default  21.000000        watch  21.000000   7.0000   7.0000
182  Default  24.000000         rest  24.000000   6.0000   6.0000
66   Default  29.000000        dream  29.000000   5.0000   5.0000
8    Default  22.000000     distance  22.000000   4.0000   4.0000
60   Default  24.000000        trace  24.000000   3.0000   3.0000
1    Default  34.000000          add  34.000000   2.0000   2.0000
214  Default  33.000000        float  33.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
45   Topic10   3.246003       expand  13.709480   2.9609  -3.2143
44   Topic10   2.070122   especially  10.961823   2.7348  -3.6641
267  Topic10   3.927144     anything  23.804340   2.5996  -3.0238
278  Topic10   3.750984        allow  24.970325   2.5059  -3.0697
24   Topic10   3.490360        organ  23.335155   2.5016  -3.1417
62   Topic10   3.137138        watch  21.119545   2.4947  -3.2484
37   Topic10   2.250913     consider  16.522375   2.4082  -3.5804
17   Topic10   2.697584          hum  22.404583   2.2847  -3.3993
88   Topic10   2.877559        state  24.584869   2.2564  -3.3348
236  Topic10   2.153504       modern  21.170319   2.1161  -3.6246
110  Topic10   1.631346       middle  17.109989   2.0513  -3.9023
26   Topic10   2.166243      pattern  25.541298   1.9343  -3.6187
47   Topic10   2.611873         hour  31.654709   1.9068  -3.4316
67   Topic10   2.543216      element  33.978367   1.8093  -3.4583
169  Topic10   1.793481       phrase  24.808668   1.7746  -3.8075
42   Topic10   2.101756     ensemble  30.964752   1.7115  -3.6489
172  Topic10   2.451187       remain  36.368942   1.7045  -3.4951
18   Topic10   0.901184   industrial  14.493943   1.6238  -4.4958
182  Topi